In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import keras
import tensorflow
from keras import Sequential
from keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics, svm
from sklearn.metrics import average_precision_score

Using TensorFlow backend.


In [44]:
df = pd.read_csv('COVID19_open_line_list.csv')

In [45]:
#df = df[pd.notnull(df['age'])]
#df = df[pd.notnull(df['sex'])]
#df = df[pd.notnull(df['country'])]
df = df[pd.notnull(df['symptoms'])]
#df = df[pd.notnull(df['lives_in_Wuhan'])]
df.drop(columns = ['city','province','latitude','longitude','geo_resolution','additional_information','source','sequence_available','outcome','notes_for_discussion','location','admin3','admin2','admin1','country_new','admin_id','data_moderator_initials'])

,ID,age,sex,country,wuhan(0)_not_wuhan(1),date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,lives_in_Wuhan,travel_history_dates,travel_history_location,reported_market_exposure,chronic_disease_binary,chronic_disease,date_death_or_discharge
30,6,NaN,NaN,China,1,NaN,NaN,24.01.2020,pneumonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,11,NaN,NaN,China,1,NaN,NaN,23.01.2020,pneumonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,10913,80-89,male,Japan,1,09.02.2020,NaN,18.02.2020,"dyspnea, fever",NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,11272,80-89,female,Japan,1,22.01.2020,01.02.2020,13.02.2020,"anorexia, fatigue",NaN,NaN,NaN,NaN,NaN,NaN,13.02.2020
245,11553,80-89,male,Japan,1,27.01.2020,19.02.2020,21.02.2020,anorexia,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12372,12552,NaN,NaN,Japan,1,16.02.2020,25.02.2020,26.02.2020,cough,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12664,12892,NaN,NaN,Nigeria,1,NaN,NaN,27.02.2020,no serious symptoms,NaN,25.02.2020,"Milan, Italy",NaN,NaN,NaN,NaN
13186,13436,NaN,female,Sweden,1,NaN,NaN,27.02.2020,similar to a respiratory infection,no,NaN,Germany,NaN,NaN,NaN,NaN
13197,13478,NaN,female,Netherlands,1,NaN,NaN,28.02.2020,mild,NaN,NaN,returned from Italy this week,NaN,NaN,NaN,NaN


In [46]:
test = df.iloc[:,13]
symptom_set = set()
for row in list(test):
    row = str(row)
    l = row.split(",")
    for symp in l:
        symp = symp.strip()
        if symp == "nan":
            continue
        elif symp not in symptom_set:
            if "fever" in symp or "Fever" in symp:
                symptom_set.add("fever")
            else:
                symptom_set.add(symp)
        else:
            continue
print(symptom_set)

{'no serious symptoms', 'anhelation', 'sore limbs', 'no symptoms', 'sweating', 'expectoration', 'chest pain', 'severe dyspnea', 'similar to a respiratory infection', 'respiratory symptoms', 'headache.', 'shortness breath', 'no respiratory symptoms', 'chest tightness', 'conjunctivitis', 'acute pharyngitis', 'wheezing', 'Sore throat', 'dry throat', 'lack of energy', 'pharyngeal discomfort', 'rhinorrhea', 'cough; sore body; cold', 'sputum', 'discomfort', 'flu-like symptoms', 'muscle aches', 'rhinorrhoea', 'Pharyngeal dryness', 'myalgias', 'mild', 'diarrheoa', 'weakness', 'muscle soreness', 'diarrhoea', 'pneumonia', 'systemic weakness', 'feeling ill', 'Discharge', 'breathing difficulty', 'acute respiratory viral infection', 'mild coughing', 'Sputum', 'rever', 'Headache', 'muscular soreness', 'asymptomatic', 'fever', 'couh', 'toothache', 'then 38 ℃)', 'other symptoms', 'Fatigue', 'headache; dry mouth', 'fatigue', 'aching muscles', 'muscle pain', 'difficulty walking', 'difficulty breathing',

In [47]:
test = df.iloc[:,5]
country_dict = dict()
count = 0
for row in list(test):
    row = str(row)
    if row in country_dict:
        continue
    else:
        country_dict[row] = count
        count+= 1
print(country_dict)

{'China': 0, 'Japan': 1, 'nan': 2, 'United States': 3, 'Thailand': 4, 'Vietnam': 5, 'South Korea': 6, 'Singapore': 7, 'Malaysia': 8, 'Spain': 9, 'Cambodia': 10, 'Shaanxi': 11, 'France': 12, 'Lebanon': 13, 'Philippines': 14, 'Germany': 15, 'Lithuania': 16, 'Nepal': 17, 'Italy': 18, 'Russia': 19, 'Belgium': 20, 'Nigeria': 21, 'Sweden': 22, 'Netherlands': 23, 'Ecuador': 24}


In [48]:
cleanedData = []
count = 0
Count = 0
for index,row in df.iterrows():
    cur = []
    Count += 1
    '''
    # Clean the age column
    if str(row["age"]).isnumeric():
        #cur.append(int(row["age"]) // 10)
        cur += [0] * (int(row["age"]) // 10) + [1] + (9 - (int(row["age"]) // 10)) * [0]
    else:
        if "-" in row["age"]:
            loc = row["age"].index("-")
            #cur.append(int(row["age"][:loc]) // 10)
            cur += [0] * (int(row["age"][:loc]) // 10) + [1] + (9 - (int(row["age"][:loc]) // 10)) * [0]
        else:
            #cur.append(int(float(row["age"]) // 10))
            cur += [0] * ((int(float(row["age"]))) // 10) + [1] + (9 - ((int(float(row["age"]))) // 10)) * [0]
            
    '''
    # Clean the sex column
    if row["sex"] == "male":
        cur += [1,0]
    else:
        cur += [0,1]
    '''
    # Clean the country
    #cur.append(country_dict[str(row["country"])])
    cur += [0]*(country_dict[str(row["country"])]) + [1] + (26 -country_dict[str(row["country"])]) * [0]
    
    # Clean the Wuhan column, check whether they are citizens in Wuhan, 1 means not Wuhan
    cur.append(row["wuhan(0)_not_wuhan(1)"])
    # Clean the sympton date
    '''
    # Clean the sympton
    temp = str(row["symptoms"]).lower()
    # fever, cough, pneumonitis, fatigue
    # Fever- related
    if "fever" in temp or "headache" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Cough related
    if "cou" in temp or "throa" in temp or "dry" in temp or "pharyngeal" in temp or "expectoration" in temp or "flu" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Cold related
    if "chill" in temp or "nose" in temp or "nasal" in temp or "sneez" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # pneumonitis related
    if "pneumon" in temp or "respiratory" in temp or "breath" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Muscle or fatigue, physical issue related
    if "fatigue" in temp or "myalgias" in temp or "musc" in temp or "walk" in temp or "chest" in temp or "limbs" in temp or "joint" in temp or "physical" in temp or "energy" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Stomacha related
    if "diarrhoea" in temp or "abdominal" in temp:
        cur.append(1)
    else:
        cur.append(0)
    # Other symptom or non-symptom
    if len(temp) > 0 and temp != "nan":
        cur.append(1)
    else:
        cur.append(0)
    
    # Clean the column whether live in Wuhan or have travel relations with Wuhan
    if str(row["lives_in_Wuhan"]).lower() == "nan" or str(row["lives_in_Wuhan"]).lower() == "no" or "wuhan" not in str(row["travel_history_location"]).lower():
        cur.append(0)
    else:
        cur.append(1)
        
    # Clean the target value: date_death_or_discharge
    if str(row["date_death_or_discharge"]).lower() == "nan":
        count += 1
        cur.append(0)
    else:
        cur.append(1)
    cleanedData.append(cur)
    
#print(count,Count)
#print(cleanedData)

In [50]:
import csv
f = open("NewCleanedConfirmedCase.csv", "a")
fieldnames = ['male','female','Fever','Cough','Cold','Pneumonitis','Fatigue','Stomacha','Other Symptoms','Relation with Wuhan','Death or Not']
writer = csv.DictWriter(f, fieldnames=fieldnames)
writer.writeheader()
writer = csv.writer(f)
writer.writerows(cleanedData)
f.close()

In [52]:
df = pd.read_csv("NewCleanedConfirmedCase.csv")

In [53]:
df

,male,female,Fever,Cough,Cold,Pneumonitis,Fatigue,Stomacha,Other Symptoms,Relation with Wuhan,Death or Not
0,0,1,0,0,0,1,0,0,1,0,0
1,0,1,0,0,0,1,0,0,1,0,0
2,1,0,1,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,1,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
489,0,1,0,1,0,0,0,0,1,0,0
490,0,1,0,0,0,0,0,0,1,0,0
491,0,1,0,0,0,1,0,0,1,0,0
492,0,1,0,0,0,0,0,0,1,0,0


In [57]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
print(X)
print(Y)

     male  female  Fever  Cough  Cold  Pneumonitis  Fatigue  Stomacha  \
0       0       1      0      0     0            1        0         0   
1       0       1      0      0     0            1        0         0   
2       1       0      1      0     0            0        0         0   
3       0       1      0      0     0            0        1         0   
4       1       0      0      0     0            0        0         0   
..    ...     ...    ...    ...   ...          ...      ...       ...   
489     0       1      0      1     0            0        0         0   
490     0       1      0      0     0            0        0         0   
491     0       1      0      0     0            1        0         0   
492     0       1      0      0     0            0        0         0   
493     0       1      1      0     0            0        0         0   

     Other Symptoms  Relation with Wuhan  
0                 1                    0  
1                 1                  

In [98]:
from sklearn.metrics import precision_recall_fscore_support
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1) 
# 80% training and 20% test
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# precision, recall ,f1, confusion matrix
precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.9501197542434656, 0.9595959595959596, 0.9530723905723905, None)

In [97]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='weighted')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.9208244056728905, 0.9595959595959596, 0.9398104758929501, None)

In [22]:
# Another dataset set with much more records
from sklearn.utils import shuffle
df = pd.read_csv("Cleaned-Data.csv")
df

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,Other
316796,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,Other
316797,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,1,Other
316798,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,Other


In [23]:
df = shuffle(df).sample(frac = 0.1)
df

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
100313,0,1,1,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,Republic of Korean
164897,1,1,1,1,0,0,1,1,0,0,...,1,0,0,1,0,0,1,0,0,Spain
2883,0,1,1,1,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,China
30277,0,1,1,1,1,0,0,1,1,1,...,0,1,1,0,0,0,0,1,0,China
268948,0,1,1,1,1,0,0,1,1,0,...,0,0,0,1,0,0,0,1,0,Other-EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55626,0,1,1,1,1,0,0,1,1,1,...,0,1,0,0,0,1,0,0,1,Italy
291887,1,1,0,0,0,0,1,1,0,0,...,1,0,0,0,1,0,1,0,0,Other
166235,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,Spain
65885,1,1,0,0,0,0,1,1,1,0,...,0,0,0,1,0,0,1,0,0,Iran


In [24]:
Y = df[["Severity_Mild","Severity_Moderate","Severity_None","Severity_Severe"]] 
Y

,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe
100313,0,1,0,0
164897,0,1,0,0
2883,0,1,0,0
30277,1,0,0,0
268948,0,1,0,0
...,...,...,...,...
55626,0,0,0,1
291887,0,0,1,0
166235,0,0,1,0
65885,0,1,0,0


In [25]:
X = df.drop(["Severity_Mild","Severity_Moderate","Severity_None","Severity_Severe","Country"], axis = 1)
X

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Contact_Dont-Know,Contact_No,Contact_Yes
100313,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
164897,1,1,1,1,0,0,1,1,0,0,...,1,0,0,0,0,1,0,1,0,0
2883,0,1,1,1,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
30277,0,1,1,1,1,0,0,1,1,1,...,0,0,0,1,0,0,1,0,1,0
268948,0,1,1,1,1,0,0,1,1,0,...,0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55626,0,1,1,1,1,0,0,1,1,1,...,0,0,1,0,0,0,1,0,0,1
291887,1,1,0,0,0,0,1,1,0,0,...,1,0,0,0,0,1,0,1,0,0
166235,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,0
65885,1,1,0,0,0,0,1,1,1,0,...,0,0,0,0,1,0,0,1,0,0


In [30]:
# To be continued............
'''
Create a model without assigning the Model Weight, and the Model has only 1 hidden layer with twice as many nodes as the input dimensionality
'''
def create_model_null():
    model = Sequential()
    model.add(Dense(128,input_dim = 22, activation = 'relu'))
    model.add(Dense(512,input_dim = 128, activation = 'relu'))
    model.add(Dense(128,input_dim = 512, activation = 'relu'))
    model.add(Dense(4,activation = 'softmax'))
    return model


'''
Model just with backpropagation
'''
def BP(X,Y):
    n = create_model_null()
    n.compile(optimizer = 'Adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    Count = 0
    # while Count < 10:
    # n.fit(X, Y, validation_split=0.2, epochs=100, batch_size=16, verbose=1, shuffle = True)
    history = n.fit(X, Y, validation_split=0.3, epochs=2000, batch_size=256, verbose=2, shuffle = True)
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    
BP(X,Y)

Train on 22176 samples, validate on 9504 samples
Epoch 1/2000
 - 0s - loss: 1.3891 - accuracy: 0.2459 - val_loss: 1.3871 - val_accuracy: 0.2489
Epoch 2/2000
 - 0s - loss: 1.3852 - accuracy: 0.2623 - val_loss: 1.3886 - val_accuracy: 0.2483
Epoch 3/2000
 - 0s - loss: 1.3835 - accuracy: 0.2719 - val_loss: 1.3886 - val_accuracy: 0.2504
Epoch 4/2000
 - 0s - loss: 1.3817 - accuracy: 0.2817 - val_loss: 1.3902 - val_accuracy: 0.2514
Epoch 5/2000
 - 0s - loss: 1.3799 - accuracy: 0.2887 - val_loss: 1.3910 - val_accuracy: 0.2521
Epoch 6/2000
 - 0s - loss: 1.3782 - accuracy: 0.2892 - val_loss: 1.3955 - val_accuracy: 0.2475
Epoch 7/2000
 - 0s - loss: 1.3765 - accuracy: 0.2941 - val_loss: 1.3931 - val_accuracy: 0.2514
Epoch 8/2000
 - 0s - loss: 1.3739 - accuracy: 0.3006 - val_loss: 1.3937 - val_accuracy: 0.2498
Epoch 9/2000
 - 0s - loss: 1.3725 - accuracy: 0.3004 - val_loss: 1.3967 - val_accuracy: 0.2469
Epoch 10/2000
 - 0s - loss: 1.3701 - accuracy: 0.3085 - val_loss: 1.3989 - val_accuracy: 0.2524


Epoch 86/2000
 - 0s - loss: 1.0913 - accuracy: 0.4821 - val_loss: 1.8272 - val_accuracy: 0.2286
Epoch 87/2000
 - 0s - loss: 1.0869 - accuracy: 0.4839 - val_loss: 1.8409 - val_accuracy: 0.2226
Epoch 88/2000
 - 0s - loss: 1.0879 - accuracy: 0.4860 - val_loss: 1.8426 - val_accuracy: 0.2294
Epoch 89/2000
 - 0s - loss: 1.0912 - accuracy: 0.4807 - val_loss: 1.8406 - val_accuracy: 0.2315
Epoch 90/2000
 - 0s - loss: 1.0822 - accuracy: 0.4863 - val_loss: 1.8256 - val_accuracy: 0.2307
Epoch 91/2000
 - 0s - loss: 1.0817 - accuracy: 0.4858 - val_loss: 1.8704 - val_accuracy: 0.2291
Epoch 92/2000
 - 0s - loss: 1.0802 - accuracy: 0.4876 - val_loss: 1.8744 - val_accuracy: 0.2286
Epoch 93/2000
 - 0s - loss: 1.0787 - accuracy: 0.4848 - val_loss: 1.8582 - val_accuracy: 0.2319
Epoch 94/2000
 - 0s - loss: 1.0777 - accuracy: 0.4834 - val_loss: 1.8590 - val_accuracy: 0.2254
Epoch 95/2000
 - 0s - loss: 1.0737 - accuracy: 0.4849 - val_loss: 1.8844 - val_accuracy: 0.2316
Epoch 96/2000
 - 0s - loss: 1.0734 - acc

Epoch 171/2000
 - 0s - loss: 1.0037 - accuracy: 0.4882 - val_loss: 2.1439 - val_accuracy: 0.2251
Epoch 172/2000
 - 0s - loss: 1.0020 - accuracy: 0.4902 - val_loss: 2.1395 - val_accuracy: 0.2310
Epoch 173/2000
 - 0s - loss: 1.0032 - accuracy: 0.4895 - val_loss: 2.1283 - val_accuracy: 0.2297
Epoch 174/2000
 - 0s - loss: 1.0000 - accuracy: 0.4890 - val_loss: 2.1661 - val_accuracy: 0.2294
Epoch 175/2000
 - 0s - loss: 1.0003 - accuracy: 0.4912 - val_loss: 2.1620 - val_accuracy: 0.2265
Epoch 176/2000
 - 0s - loss: 1.0047 - accuracy: 0.4844 - val_loss: 2.1550 - val_accuracy: 0.2261
Epoch 177/2000
 - 0s - loss: 1.0005 - accuracy: 0.4884 - val_loss: 2.1449 - val_accuracy: 0.2293
Epoch 178/2000
 - 0s - loss: 1.0026 - accuracy: 0.4835 - val_loss: 2.1554 - val_accuracy: 0.2272
Epoch 179/2000
 - 0s - loss: 0.9999 - accuracy: 0.4900 - val_loss: 2.1458 - val_accuracy: 0.2290
Epoch 180/2000
 - 0s - loss: 0.9967 - accuracy: 0.4928 - val_loss: 2.1814 - val_accuracy: 0.2297
Epoch 181/2000
 - 0s - loss: 1

Epoch 256/2000
 - 0s - loss: 0.9751 - accuracy: 0.4883 - val_loss: 2.2614 - val_accuracy: 0.2312
Epoch 257/2000
 - 0s - loss: 0.9743 - accuracy: 0.4903 - val_loss: 2.2645 - val_accuracy: 0.2282
Epoch 258/2000
 - 0s - loss: 0.9736 - accuracy: 0.4932 - val_loss: 2.2562 - val_accuracy: 0.2284
Epoch 259/2000
 - 0s - loss: 0.9746 - accuracy: 0.4895 - val_loss: 2.2602 - val_accuracy: 0.2259
Epoch 260/2000
 - 0s - loss: 0.9746 - accuracy: 0.4895 - val_loss: 2.2554 - val_accuracy: 0.2318
Epoch 261/2000
 - 0s - loss: 0.9740 - accuracy: 0.4899 - val_loss: 2.2793 - val_accuracy: 0.2287
Epoch 262/2000
 - 0s - loss: 0.9729 - accuracy: 0.4870 - val_loss: 2.2672 - val_accuracy: 0.2243
Epoch 263/2000
 - 0s - loss: 0.9763 - accuracy: 0.4833 - val_loss: 2.2501 - val_accuracy: 0.2306
Epoch 264/2000
 - 0s - loss: 0.9731 - accuracy: 0.4882 - val_loss: 2.2765 - val_accuracy: 0.2265
Epoch 265/2000
 - 0s - loss: 0.9731 - accuracy: 0.4914 - val_loss: 2.2761 - val_accuracy: 0.2325
Epoch 266/2000
 - 0s - loss: 0

Epoch 341/2000
 - 0s - loss: 0.9594 - accuracy: 0.4896 - val_loss: 2.3273 - val_accuracy: 0.2257
Epoch 342/2000
 - 0s - loss: 0.9589 - accuracy: 0.4887 - val_loss: 2.3365 - val_accuracy: 0.2307
Epoch 343/2000
 - 0s - loss: 0.9585 - accuracy: 0.4906 - val_loss: 2.3304 - val_accuracy: 0.2281
Epoch 344/2000
 - 0s - loss: 0.9587 - accuracy: 0.4903 - val_loss: 2.3181 - val_accuracy: 0.2265
Epoch 345/2000
 - 0s - loss: 0.9571 - accuracy: 0.4903 - val_loss: 2.3353 - val_accuracy: 0.2292
Epoch 346/2000
 - 0s - loss: 0.9576 - accuracy: 0.4932 - val_loss: 2.3387 - val_accuracy: 0.2250
Epoch 347/2000
 - 0s - loss: 0.9582 - accuracy: 0.4893 - val_loss: 2.3337 - val_accuracy: 0.2274
Epoch 348/2000
 - 0s - loss: 0.9584 - accuracy: 0.4905 - val_loss: 2.3345 - val_accuracy: 0.2274
Epoch 349/2000
 - 0s - loss: 0.9563 - accuracy: 0.4899 - val_loss: 2.3384 - val_accuracy: 0.2262
Epoch 350/2000
 - 0s - loss: 0.9576 - accuracy: 0.4899 - val_loss: 2.3251 - val_accuracy: 0.2263
Epoch 351/2000
 - 0s - loss: 0

Epoch 426/2000
 - 0s - loss: 0.9473 - accuracy: 0.4909 - val_loss: 2.3964 - val_accuracy: 0.2271
Epoch 427/2000
 - 0s - loss: 0.9454 - accuracy: 0.4916 - val_loss: 2.3801 - val_accuracy: 0.2255
Epoch 428/2000
 - 0s - loss: 0.9486 - accuracy: 0.4897 - val_loss: 2.4052 - val_accuracy: 0.2249
Epoch 429/2000
 - 0s - loss: 0.9465 - accuracy: 0.4921 - val_loss: 2.4162 - val_accuracy: 0.2265
Epoch 430/2000
 - 0s - loss: 0.9467 - accuracy: 0.4894 - val_loss: 2.4017 - val_accuracy: 0.2297
Epoch 431/2000
 - 0s - loss: 0.9468 - accuracy: 0.4932 - val_loss: 2.4031 - val_accuracy: 0.2304
Epoch 432/2000
 - 0s - loss: 0.9469 - accuracy: 0.4920 - val_loss: 2.4109 - val_accuracy: 0.2283
Epoch 433/2000
 - 0s - loss: 0.9485 - accuracy: 0.4914 - val_loss: 2.3760 - val_accuracy: 0.2295
Epoch 434/2000
 - 0s - loss: 0.9458 - accuracy: 0.4927 - val_loss: 2.4028 - val_accuracy: 0.2250
Epoch 435/2000
 - 0s - loss: 0.9468 - accuracy: 0.4907 - val_loss: 2.3993 - val_accuracy: 0.2247
Epoch 436/2000
 - 0s - loss: 0

KeyboardInterrupt: 